### Insatll and Import libraries

In [12]:
# pip install openmeteo-requests
# pip install requests-cache 
# pip install retry-requests

In [1]:
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry

### Define relevant Date

In [10]:
date = "2023-12-01"

### Define outputfile

In [17]:
outputfile = "./02_weather_forecaste.csv"

### Setup the Open-Meteo API client

In [12]:
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

url = "https://api.open-meteo.com/v1/forecast" # URL API

params = {
	"latitude": 47.4239,
	"longitude": 9.3748,
	"daily": ["temperature_2m_max", "temperature_2m_min", "rain_sum", "snowfall_sum"],
	"start_date": date,
	"end_date": date
}

responses = openmeteo.weather_api(url, params=params)

### Process location

In [13]:
response = responses[0]
print(f"Coordinates {response.Latitude()}°E {response.Longitude()}°N")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

Coordinates 47.439998626708984°E 9.399999618530273°N
Elevation 679.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


### Process data for requested day

In [20]:
daily = response.Daily()
daily_temperature_2m_max = daily.Variables(0).ValuesAsNumpy()
daily_temperature_2m_min = daily.Variables(1).ValuesAsNumpy()
daily_rain_sum = daily.Variables(2).ValuesAsNumpy()
daily_snowfall_sum = daily.Variables(3).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s"),
	end = pd.to_datetime(daily.TimeEnd(), unit = "s"),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
)}

daily_data["temperature_2m_max"] = daily_temperature_2m_max
daily_data["temperature_2m_min"] = daily_temperature_2m_min
daily_data["rain_sum"] = daily_rain_sum
daily_data["snowfall_sum"] = daily_snowfall_sum

daily_dataframe = pd.DataFrame(data = daily_data)
daily_dataframe.head()

date                  datetime64[ns]
temperature_2m_max           float32
temperature_2m_min           float32
rain_sum                     float32
snowfall_sum                 float32
dtype: object

In [18]:
daily_dataframe.to_csv(outputfile, sep=";", index=False)